In [2]:
#테스트 세트를 사용해 성능을 확인하다 보면 점점 테스트 세트에 맞추게 된다.
#테스트 세트를 최대한 덜 사용하면서 max_depth 매개변수를 사용한 하이퍼파라미터 튜닝하기 !

In [4]:
#훈련 세트의 20%를 검증 세트로 만들자 !

In [6]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [8]:
data = wine[['alcohol', 'sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

In [10]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [12]:
#훈련 세트를 훈련 세트와 검증 세트로 나누기
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

In [14]:
#훈련 세트와 검증 세트의 크기 확인하기
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [18]:
#모델 만들고 평가하기
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

#훈련 세트에 과대적합 되어있는 상태

0.9971133028626413
0.864423076923077


In [20]:
#검증 세트를 만드느라 훈련 세트가 줄어든다 → 많은 데이터를 훈련에 사용해야 좋은 모델이 만들어진다. ㅠ
#그렇다고 검증 세트를 조금 떼어 놓으면 검증 점수가 불안정 해진다.
#→ 교차 검증 이용! (검증 세트를 떼어 내어 평가하는 과정을 여러 번 반복)
#반복하여 나온 점수를 평균하여 최종 검증 점수를 얻는다.

In [22]:
#교차 검증
#직접 검증 세트를 떼어내지 않고 훈련 세트 전체를 전달
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00835609, 0.00675201, 0.00601315, 0.00548887, 0.00490403]), 'score_time': array([0.00134873, 0.00063014, 0.00047064, 0.00041819, 0.00054693]), 'test_score': array([0.87019231, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [24]:
#cross_validate() 함수는 fit_time, score_time, test_score 키를 가진 딕셔너리를 반환한다. 
#처음 2개의 키는 각각 모델을 훈련하는 시간과 검증하는 시간을 의미한다. 각 키마다 5개의 숫자가 담겨 있는 것을 보아 이 함수는 기본적으로 5-폴드 교차 검증을 수행함을 알 수 있다.
#교차 검증의 최종 점수는 test_score 키에 담긴 5개의 점수를 평균하여 얻을 수 있다. 이름은 test_score지만 검증 폴드의 점수이다.

In [26]:
import numpy as np
print(np.mean(scores['test_score']))

0.8554925223957948


In [28]:
#cv 매개변수에서 폴드 수를 바꿀 수도 있다.
#cross_validate()는 훈련 세트를 섞어 폴드를 나누지 않는다. 앞서 우리는 train_test_split() 함수로 전체 데이터를 섞은 후 훈련 세트를 준비했기 때문에 따로 섞을 필요가 없긴 하다.
#하지만 만약 교차 검증을 할 때 훈련 세트를 섞고 싶다면 분할기를 지정해야 한다.
#분류 모델일 경우 StratifiedKFold 분할기를 사용한다.

In [32]:
#앞서 수행한 교차 검증은 다음 코드와 동일하다.
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.8554925223957948


In [34]:
#10폴드 교차 검증 수행 시 ..
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8581873425226026


In [36]:
#매개변수 값을 바꿔가며 교차검증을 통해 가장 좋은 성능이 나오는 모델 찾기!

In [38]:
#사이킷런에서 제공하는 그리드 서치 클래스는 하이퍼파라미터 탐색과 교차 검증을 한 번에 수행한다.
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease':[0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [40]:
#그리드 서치 클래스에 탐색 대상 모델과 변수 전달하여 객체 만든다.
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
#n_jobs 변수는 병렬 실행에 사용할 CPU 코어 수를 지정한다. (-1은 모든 코어를 사용함을 의미)

In [42]:
#5x5(cv 기본값)=25개의 모델 훈련
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [44]:
#그리드 서치는 훈련이 끝나면 교차 검증 점수가 가장 높은 모델의 매개변수 조합으로 전체 훈련 세트에서 자동으로 다시 모델을 훈련한다.
#이 모델은 gs 객체의 best_estimator_ 속성에 저장되어 있다.
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [48]:
#최적의 매개변수는 best_params_ 속성에 저장
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [50]:
#각 매개변수에서 수행한 교차검증의 평균 점수
print(gs.cv_results_['mean_test_score'])

[0.86800067 0.86453617 0.86492226 0.86780891 0.86761605]


In [52]:
#argmax() 함수를 사용하면 가장 큰 값의 인덱스를 추출할 수 있다.
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [54]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001), #0.0001에서 0.001이 되기까지 0.0001씩 더한다.
          'max_depth': range(5,20,1),
          'min_samples_split' : range(2,100,10)
         }
#9x15x10=1,350 -> 5-폴드 교차검증 수행하므로 1,350x5=6,750개의 모델 ..

In [56]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42),params, n_jobs=-1)
gs.fit(train_input, train_target)

/opt/anaconda3/lib/python3.12/site-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [58]:
#최상의 매개변수 조합 확인
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [60]:
#최상의 교차검증 점수 확인
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


In [62]:
#매개변수의 간격을 어떻게 정할 것인가? -> 랜덤 서치 !
#랜덤 서치는 매개변수를 샘플링할 수 있는 확률 분포 객체를 전달한다.

In [64]:
from scipy.stats import uniform, randint #균등분포 (uniform은 실수, randint는 정수)

In [66]:
rgen = randint(0,10)
rgen.rvs(10)

array([4, 8, 2, 7, 6, 8, 9, 7, 5, 5])

In [68]:
np.unique(rgen.rvs(1000), return_counts=True)
#어느정도 고르게 추출됨을 볼 수 있다.

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([106, 102,  81, 104,  93,  91,  93, 114, 106, 110]))

In [70]:
ugen = uniform(0,1)
ugen.rvs(10)

array([0.41208546, 0.97208937, 0.39948989, 0.19311555, 0.04962224,
       0.65421061, 0.56383224, 0.63151352, 0.60474737, 0.86634671])

In [80]:
params = {'min_impurity_decrease' : uniform(0.0001, 0.001),
          'max_depth': randint(20,50),
          'min_samples_split': randint(2,25),
          'min_samples_leaf': randint(1,25),
         }

In [84]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42) #100개의 매개변수 조합 생성
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x168b80cb0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x168b82540>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x168b82840>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x168b82de0>},
                   random_state=42)

In [86]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [88]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [90]:
dt = gs.best_estimator_ #최적의 모델 저장
print(dt.score(test_input, test_target))

#최적의 모델을 최종 모델로 지정 후 테스트 세트 성능 확인

0.86
